In [19]:
#import and load file

In [20]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [21]:
#preprocess data

In [22]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))


In [23]:
#Create training and testing data

In [24]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [25]:
#building model

In [26]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
47/47 [==============================] - 0s 5ms/step - loss: 2.2161 - accuracy: 0.1064
Epoch 2/200
47/47 [==============================] - 0s 511us/step - loss: 2.2291 - accuracy: 0.1277
Epoch 3/200
47/47 [==============================] - 0s 511us/step - loss: 2.1163 - accuracy: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 681us/step - loss: 2.1136 - accuracy: 0.1702
Epoch 5/200
47/47 [==============================] - 0s 680us/step - loss: 2.1469 - accuracy: 0.1489
Epoch 6/200
47/47 [==============================] - 0s 681us/step - loss: 2.0517 - accuracy: 0.3617
Epoch 7/200
47/47 [==============================] - 0s 681us/step - loss: 1.9790 - accuracy: 0.3617
Epoch 8/200
47/47 [==============================] - 0s 681us/step - loss: 1.9520 - accuracy: 0.4043
Epoch 9/200
47/47 [==============================] - 0s 870us/step - loss: 1.9520 - accuracy: 0.3404
Epoch 10/200
47/47 [==============================] - 0s 681us/step - loss: 1.8039 - accuracy

47/47 [==============================] - 0s 511us/step - loss: 0.0362 - accuracy: 1.0000
Epoch 82/200
47/47 [==============================] - 0s 681us/step - loss: 0.1347 - accuracy: 0.9787
Epoch 83/200
47/47 [==============================] - 0s 681us/step - loss: 0.0511 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 510us/step - loss: 0.0968 - accuracy: 0.9787
Epoch 85/200
47/47 [==============================] - 0s 851us/step - loss: 0.1280 - accuracy: 0.9787
Epoch 86/200
47/47 [==============================] - 0s 681us/step - loss: 0.0627 - accuracy: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 851us/step - loss: 0.1467 - accuracy: 0.9574
Epoch 88/200
47/47 [==============================] - 0s 510us/step - loss: 0.0705 - accuracy: 1.0000
Epoch 89/200
47/47 [==============================] - 0s 510us/step - loss: 0.0789 - accuracy: 1.0000
Epoch 90/200
47/47 [==============================] - 0s 511us/step - loss: 0.0561 - accuracy: 

47/47 [==============================] - 0s 681us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 161/200
47/47 [==============================] - 0s 511us/step - loss: 0.0500 - accuracy: 0.9787
Epoch 162/200
47/47 [==============================] - 0s 511us/step - loss: 0.0088 - accuracy: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 511us/step - loss: 0.0177 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 510us/step - loss: 0.0325 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 511us/step - loss: 0.0297 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 511us/step - loss: 0.0141 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 511us/step - loss: 0.0090 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 340us/step - loss: 0.0286 - accuracy: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 511us/step - loss: 0.0358 - a

In [27]:
#prediction

In [28]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):

    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                
    return(np.array(bag))
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [29]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [30]:
chatbot_response("Hi, there")

'Hi there, how can I help?'